In [1]:
from insilicoICH.study import ICHStudy

/scratch/brandon.nelson/demos/pediatric_ich_cadt/.demo/lib/python3.13/site-packages/monai/utils/module.py:399: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  pkg = __import__(module)  # top level module


In [2]:
from VITools import get_available_phantoms
get_available_phantoms()

{'Water Phantom': VITools.examples.WaterPhantom,
 '0.0 yr UNC Head': functools.partial(<class 'insilicoICH.phantoms.head_phantoms.UNC_Head'>, age=0.0, phantom_dir=PosixPath('/projects01/didsr-aiml/brandon.nelson/pedsilicoICH/phantoms/UNC_Head_phantom')),
 '1.0 yr UNC Head': functools.partial(<class 'insilicoICH.phantoms.head_phantoms.UNC_Head'>, age=1.0, phantom_dir=PosixPath('/projects01/didsr-aiml/brandon.nelson/pedsilicoICH/phantoms/UNC_Head_phantom')),
 '2.0 yr UNC Head': functools.partial(<class 'insilicoICH.phantoms.head_phantoms.UNC_Head'>, age=2.0, phantom_dir=PosixPath('/projects01/didsr-aiml/brandon.nelson/pedsilicoICH/phantoms/UNC_Head_phantom')),
 '6.5 yr NIHPD Head': functools.partial(<class 'insilicoICH.phantoms.head_phantoms.NIHPD_Head'>, age=6.5, phantom_dir=PosixPath('/projects01/didsr-aiml/brandon.nelson/pedsilicoICH/phantoms/NIHPD_Head_Phantom')),
 '9.0 yr NIHPD Head': functools.partial(<class 'insilicoICH.phantoms.head_phantoms.NIHPD_Head'>, age=9.0, phantom_dir=Pos

In [3]:
lesion_volume = dict(IPH=[0.2, 10], SDH=[0.2, 10], EDH=[0.2, 10], Fracture=[0, 1]) # placeholders for fracture, eventually put in length, thickness etc
lesion_attn = dict(IPH=[40, 80], SDH=[40, 80], EDH=[40, 80], Fracture=[0, 1]) # placeholders for fracture, eventually put in length, thickness etc

In [4]:
import pandas as pd
from VITools import available_scanners

print(available_scanners)
study = ICHStudy()
study.metadata = pd.concat([
    study.metadata,
    ICHStudy.generate_from_distributions(get_available_phantoms(),
                            lesion_volume=lesion_volume,
                            lesion_attenuation=lesion_attn,
                            subtype=['Fracture'],
                            fracture_length=[150, 250],
                            views=1000,
                            scanner_model=['Siemens_DefinitionFlash'],
                            study_count=5,
                            seed=88)])
study.metadata

['GE_Lightspeed16', 'GE_Lightspeed64', 'Scanner_Default', 'Siemens_DefinitionFlash']


,case_id,phantom,scanner_model,kVp,mA,pitch,views,scan_coverage,recon_kernel,slice_thickness,...,output_directory,remove_raw,subtype,lesion_volume,lesion_attenuation,edema,age,mass_effect,fracture_length,add_augmentation
0,case_0000,10.5 yr NIHPD Head,Siemens_DefinitionFlash,120.0,300.0,0.0,688.0,dynamic,soft,1,...,/scratch/brandon.nelson/demos/pediatric_ich_ca...,True,Fracture,0.202020,0.707071,0,10.5,0.650808,237,True
1,case_0001,2.0 yr UNC Head,Siemens_DefinitionFlash,120.0,300.0,0.0,872.0,dynamic,soft,1,...,/scratch/brandon.nelson/demos/pediatric_ich_ca...,True,Fracture,0.666667,0.121212,0,2.0,0.238263,218,True
2,case_0002,38.0 yr MIDA Head,Siemens_DefinitionFlash,120.0,300.0,0.0,688.0,dynamic,soft,1,...,/scratch/brandon.nelson/demos/pediatric_ich_ca...,True,Fracture,0.313131,0.202020,0,38.0,0.148469,221,True
3,case_0003,12.0 yr NIHPD Head,Siemens_DefinitionFlash,120.0,300.0,0.0,494.0,dynamic,soft,1,...,/scratch/brandon.nelson/demos/pediatric_ich_ca...,True,Fracture,0.676768,0.616162,0,12.0,0.268125,186,True
4,case_0004,11.5 yr NIHPD Head,Siemens_DefinitionFlash,120.0,300.0,0.0,662.0,dynamic,soft,1,...,/scratch/brandon.nelson/demos/pediatric_ich_ca...,True,Fracture,0.858586,0.272727,0,11.5,0.829402,157,True
